# 🚢 Titanic — Preprocessing + Classificazione  
Obiettivo: predire chi è sopravvissuto al Titanic usando le feature disponibili.  
In questo notebook eseguiamo:

1. Caricamento del dataset  
2. Pulizia e preprocessing  
3. Encoding delle variabili categoriche  
4. Train/Test split  
5. Standardizzazione  
6. Addestramento con Random Forest  
7. Valutazione  

La classificazione è stata realizzata seguendo un workflow standard di Machine Learning supervisionato, composto dai 7 passaggi fondamentali.

In [16]:
# Import librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

sns.set(style="whitegrid")

In [17]:
# Caricamento dataset Titanic
df = pd.read_csv("titanic.csv")
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22,1,0,7.25,S
1,1,1,female,38,1,0,71.28,C
2,1,3,female,26,0,0,7.92,S
3,1,1,female,35,1,0,53.10,S
4,0,3,male,35,0,0,8.05,S


In [18]:
# Selezione delle feature: classe del biglietto, sesso, età, costo del biglietto, porto d'imbarco
# Queste sono le variabili che il modello usa per imparare a predire se un passeggero è sopravvissuto o no
# Il modello deve predire: Survived  → 0 = non sopravvissuto, 1 = sopravvissuto

features = ["Pclass", "Sex", "Age", "Fare", "Embarked"]
target = "Survived"

df = df[features + [target]]
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    16 non-null     int64  
 1   Sex       16 non-null     str    
 2   Age       16 non-null     int64  
 3   Fare      16 non-null     float64
 4   Embarked  16 non-null     str    
 5   Survived  16 non-null     int64  
dtypes: float64(1), int64(3), str(2)
memory usage: 900.0 bytes


In [19]:
# Preprocessing dei dati
# Gestione dei valori mancanti

# Età: sostituiamo i NaN con la mediana
df["Age"] = df["Age"].fillna(df["Age"].median())

# Embarked: sostituiamo i NaN con la modalità
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

In [20]:
# Encoding delle variabili categoriche

# Sex e Embarked sono stringhe categoriche → convertite in numeri
le_sex = LabelEncoder()
le_embarked = LabelEncoder()

df["Sex"] = le_sex.fit_transform(df["Sex"])
df["Embarked"] = le_embarked.fit_transform(df["Embarked"])

# Esempio: Sex: male → 1, female → 0 e Embarked: S → 2, C → 0, Q → 1


In [21]:
# Preparazione x e y
X = df.drop("Survived", axis=1)
y = df["Survived"]

X.head()

,Pclass,Sex,Age,Fare,Embarked
0,3,1,22,7.25,2
1,1,0,38,71.28,0
2,3,0,26,7.92,2
3,1,0,35,53.10,2
4,3,1,35,8.05,2


In [22]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [23]:
# Standardizzazione che trasforma ogni feature in: media = 0, deviazione standard = 1. Questo aiuta molti modelli a convergere meglio.

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
# Addestramento con Random Forest
# Come funziona in pratica: - Costruisce 100 alberi decisionali diversi - Ogni albero vota “sopravvissuto” o “non sopravvissuto” - La previsione finale è la maggioranza dei voti
# È un modello molto robusto, ottimo per dataset con feature miste

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [25]:
# Valutazione
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



## ✅ Conclusioni

- Il dataset Titanic è stato preprocessato correttamente  
- I missing values sono stati gestiti in modo robusto  
- Le variabili categoriche sono state codificate  
- Il modello Random Forest ha ottenuto buone performance  
- Il flusso è pronto per essere esteso con tuning, cross-validation o pipeline  